In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# PAGE CONFIG 
st.set_page_config(
    page_title="RoadSafe Analytics - Milestone 3",
    layout="wide"
)

st.title("🚦 RoadSafe Analytics – Milestone 3 Dashboard")
st.markdown(" ## Geospatial Analysis & Insight Extraction")

# LOAD DATA 
@st.cache_data
def load_data():
    df = pd.read_csv(r"C:\Users\acer\OneDrive\Desktop\DS_INTERN\US_Accidents_Cleaned_dataset.csv")
    df["Start_Time"] = pd.to_datetime(df["Start_Time"])
    df["Hour"] = df["Start_Time"].dt.hour
    return df

df = load_data()

# SIDEBAR FILTERS 
st.sidebar.header("🔎 Filters")

state_filter = st.sidebar.multiselect(
    "Select State",
    options=df["State"].unique(),
    default=df["State"].unique()[:5]
)

severity_filter = st.sidebar.multiselect(
    "Select Severity",
    options=sorted(df["Severity"].unique()),
    default=sorted(df["Severity"].unique())
)

filtered_df = df[
    (df["State"].isin(state_filter)) &
    (df["Severity"].isin(severity_filter))
]

# KPI METRICS
st.markdown("## 📊 Key Metrics")
col1, col2, col3 = st.columns(3)

col1.metric("Total Accidents", f"{filtered_df.shape[0]:,}")
col2.metric("Avg Severity", round(filtered_df["Severity"].mean(), 2))
col3.metric("States Selected", len(state_filter))

# GEO MAP 
st.markdown("## 🗺️ Accident Hotspots Map")

map_fig = px.scatter_mapbox(
    filtered_df.sample(10000),
    lat="Start_Lat",
    lon="Start_Lng",
    color="Severity",
    zoom=3,
    height=500,
    mapbox_style="open-street-map",
    title="Accident Hotspots by Severity"
)

st.plotly_chart(map_fig, use_container_width=True)

# TOP STATES 
st.markdown("## 🏆 Top 5 Accident-Prone States")

top_states = (
    filtered_df["State"]
    .value_counts()
    .head(5)
    .reset_index()
)

top_states.columns = ["State", "Accidents"]

fig, ax = plt.subplots()
sns.barplot(data=top_states, x="State", y="Accidents", ax=ax)
ax.set_title("Top 5 States with Highest Accidents")

st.pyplot(fig)

# TOP CITIES 
st.markdown("## 🏙️ Top 10 Accident-Prone Cities")

city_df = filtered_df.dropna(subset=["City"])
city_df = city_df[city_df["City"].str.lower() != "unknown"]

top_cities = (
    city_df["City"]
    .value_counts()
    .head(10)
    .reset_index()
)

top_cities.columns = ["City", "Accidents"]

fig_city, ax_city = plt.subplots(figsize=(8, 4))
sns.barplot(
    data=top_cities,
    x="Accidents",
    y="City",
    ax=ax_city
)

ax_city.set_title("Top 10 Cities with Highest Number of Accidents")
ax_city.set_xlabel("Number of Accidents")
ax_city.set_ylabel("City")

st.pyplot(fig_city)


# SEVERITY VS WEATHER
st.markdown("## 🌧️ Severity vs Weather Conditions")

weather_focus = filtered_df[
    filtered_df["Weather_Condition"].isin(["Rain", "Fog", "Clear"])
]

fig2, ax2 = plt.subplots()
sns.boxplot(
    data=weather_focus,
    x="Weather_Condition",
    y="Severity",
    ax=ax2
)
ax2.set_title("Accident Severity under Different Weather Conditions")

st.pyplot(fig2)

# VISIBILITY VS SEVERITY 
st.markdown("## 🌫️ Visibility Impact on Severity")

fig3, ax3 = plt.subplots()
sns.scatterplot(
    data=filtered_df.sample(5000),
    x="Visibility(mi)",
    y="Severity",
    alpha=0.4,
    ax=ax3
)
ax3.set_title("Visibility vs Accident Severity")

st.pyplot(fig3)

# TIME-BASED INSIGHT 
st.markdown("## ⏰ Accident Frequency by Time of Day")

hourly_accidents = filtered_df["Hour"].value_counts().sort_index()

fig4, ax4 = plt.subplots()
hourly_accidents.plot(kind="line", ax=ax4)
ax4.set_xlabel("Hour of Day")
ax4.set_ylabel("Number of Accidents")
ax4.set_title("Accidents by Hour")

st.pyplot(fig4)

# INSIGHTS
st.markdown("## 🧠 Key Insights (Milestone 3)")

st.success("""
✔ Accident hotspots are concentrated in specific states and urban areas  
✔ Higher severity is observed during Rain and Fog conditions  
✔ Poor visibility shows a weak but noticeable correlation with severity  
✔ Peak accident frequency occurs during morning and evening rush hours  
""")


Writing app.py


In [18]:
!python -m streamlit run app.py

^C
